<a href="https://colab.research.google.com/github/BrunoWLP/Atividades-Carnaval/blob/main/Carnaval_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Precisamos escolher a trilha sonora do nosso carnaval! Utilizando o banco de dados music-store.db, vamos tentar encontrar as melhores músicas para os nossos convidados.
Bloco de Dados

Quantas compras estão acima da média?
Vamos fazer uma playlist com as 12 músicas mais vendidas! Quais foram elas? Quais os generos e os artistas?

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/music-store.db')
cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute("SELECT * FROM sqlite_master WHERE type = 'table'"):
  print(row[1:])
  #cur.fechtall() também pode ser usado de alguma maneira pra retornar os nomes de tabelas

# Be sure to close the connection
# con.close()

('sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)')
('artists', 'artists', 2, 'CREATE TABLE artists\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    name VARCHAR(120)\n)')
('albums', 'albums', 4, 'CREATE TABLE albums\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    title VARCHAR(160)  NOT NULL,\n    artist_id INTEGER  NOT NULL,\n    FOREIGN KEY (artist_id) REFERENCES artists (id)\n    ON DELETE NO ACTION ON UPDATE NO ACTION\n)')
('employees', 'employees', 5, 'CREATE TABLE employees\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    last_name VARCHAR(20)  NOT NULL,\n    first_name VARCHAR(20)  NOT NULL,\n    title VARCHAR(30),\n    reports_to INTEGER,\n    birth_date TIMESTAMP,\n    hire_date TIMESTAMP,\n    address VARCHAR(70),\n    city VARCHAR(40),\n    state VARCHAR(40),\n    country VARCHAR(40),\n    postal_code VARCHAR(10),\n    phone VARCHAR(24),\n    fax VARCHAR(24),\n    email VARCHAR(60),\n    FOREIGN KEY (reports_to) REFERENCES employees (i

## Função para rodar queries
### A rotina para obter os dataframes via query será a mesma, então ter uma função com o molde já realizado acima agiliza o processo.

In [ ]:
def query(text):
  
  con = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/music-store.db')
  # cur = con.cursor()
  df = pd.read_sql(text, con)
  con.close()
  return df # Vai retornar um head de 3 itens para ilustrar a tabela a ser trabalhada


## Função para conferir o schema
#### Ficar conferindo o nome das tabelas e seus registros a partir do bloco lá de cima é inaceitável.<br> A partir da primeira função criamos um objeto que lê a query **SELECT * FROM sqlite_master WHERE type = 'table'**. Usando esse objeto na nova função, temos acesso aos conteúdos de qualquer tabela de forma bastante apresentável.

In [ ]:
schema = query('select * from sqlite_master where type = "table"')
schema

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,artists,artists,2,CREATE TABLE artists\n(\n id INTEGER PRIMAR...
2,table,albums,albums,4,CREATE TABLE albums\n(\n id INTEGER PRIMARY...
3,table,employees,employees,5,CREATE TABLE employees\n(\n id INTEGER PRIM...
4,table,customers,customers,6,CREATE TABLE customers\n(\n id INTEGER PRIM...
5,table,genres,genres,7,CREATE TABLE genres\n(\n id INTEGER PRIMARY...
6,table,invoices,invoices,8,CREATE TABLE invoices\n(\n id INTEGER PRIMA...
7,table,media_types,media_types,9,CREATE TABLE media_types\n(\n id INTEGER PR...
8,table,tracks,tracks,10,CREATE TABLE tracks\n(\n id INTEGER PRIMARY...
9,table,invoice_lines,invoice_lines,11,CREATE TABLE invoice_lines\n(\n id INTEGER ...


In [ ]:
def show_schema(tabela):
  '''
  :param tabela: string.
  :returns: schema do nome da tabela informada.
  '''
  for i in (schema.loc[schema['tbl_name'] == tabela, 'sql']):
      print(i)

In [ ]:
show_schema('invoice_lines')

CREATE TABLE invoice_lines
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    invoice_id INTEGER  NOT NULL,
    track_id INTEGER  NOT NULL,
    unit_price NUMERIC(10,2)  NOT NULL,
    quantity INTEGER  NOT NULL,
    FOREIGN KEY (invoice_id) REFERENCES invoices (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY (track_id) REFERENCES tracks (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)


## Qual artista possui mais albuns?

In [ ]:
data = query('''SELECT ar.id, ar.name, count(title) total_albums
            FROM artists ar 
            INNER JOIN albums alb ON ar.id = alb.artist_id
            GROUP BY name
            ORDER BY total_albums DESC''')
data.head()


,id,name,total_albums
0,90,Iron Maiden,21
1,22,Led Zeppelin,14
2,58,Deep Purple,11
3,50,Metallica,10
4,150,U2,10


A partir de: data = query('''SELECT ar.id, ar.name, title
                          FROM artists ar 
                          INNER JOIN albums alb ON ar.id = alb.artist_id''')<br>Em Python:

In [ ]:
most_alb = data.groupby('name', as_index=False).agg({'title' : 'count'}).sort_values(by='title', ascending=False)
most_alb

,name,title
91,Iron Maiden,21
107,Led Zeppelin,14
54,Deep Purple,11
125,Metallica,10
194,U2,10
...,...,...
80,Gonzaguinha,1
83,Gustav Mahler,1
84,Göteborgs Symfoniker & Neeme Järvi,1
85,Habib Koité and Bamada,1


#### DÁ-LHE IRON!

## Existem clientes de quantos países?

In [ ]:
data = query('SELECT * FROM customers')
data.head()


,id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [ ]:
data.country.unique().size

24

In [ ]:
# Outra maneira
unique_countries = data.country.drop_duplicates()
unique_countries.count()

24

#### Chegaríamos à mesma resposta com SQL com a query:

In [ ]:
data = query('''SELECT COUNT(DISTINCT country) '#_unique_countries'
            FROM customers
            ''')
data

,#_unique_countries
0,24


## Quantos clientes são brasileiros?

In [ ]:
data = query('SELECT * FROM customers')
data.head()

,id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [ ]:
data.loc[data['country'] == 'Brazil'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 12
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5 non-null      int64 
 1   first_name      5 non-null      object
 2   last_name       5 non-null      object
 3   company         4 non-null      object
 4   address         5 non-null      object
 5   city            5 non-null      object
 6   state           5 non-null      object
 7   country         5 non-null      object
 8   postal_code     5 non-null      object
 9   phone           5 non-null      object
 10  fax             5 non-null      object
 11  email           5 non-null      object
 12  support_rep_id  5 non-null      int64 
dtypes: int64(2), object(11)
memory usage: 560.0+ bytes


In [ ]:
data1 = query('SELECT count(country) Brazilians FROM customers WHERE country = "Brazil"')
data1

,Brazilians
0,5


#### Apenas 5 clientes são brasileiros

## Qual genero possui mais músicas?

In [ ]:
data = query('''SELECT genres.name genre, tracks.name track 
            FROM genres 
            INNER JOIN tracks ON genres.id = tracks.genre_id''')
data

,genre,track
0,Rock,For Those About To Rock (We Salute You)
1,Rock,Balls to the Wall
2,Rock,Fast As a Shark
3,Rock,Restless and Wild
4,Rock,Princess of the Dawn
...,...,...
3498,Classical,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...
3499,Classical,"String Quartet No. 12 in C Minor, D. 703 ""Quar..."
3500,Classical,"L'orfeo, Act 3, Sinfonia (Orchestra)"
3501,Classical,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


In [ ]:
popular_genre = data.groupby('genre', as_index=False).count().sort_values(by='track', ascending=False)
popular_genre

,genre,track
18,Rock,1297
12,Latin,579
13,Metal,374
1,Alternative & Punk,332
11,Jazz,130
23,TV Shows,93
2,Blues,81
4,Classical,74
6,Drama,64
16,R&B/Soul,61


In [ ]:
data = query('''SELECT genres.name genre, count(tracks.name) track_qty
            FROM genres 
            INNER JOIN tracks ON genres.id = tracks.genre_id
            GROUP BY genre
            ORDER BY track_qty DESC
            LIMIT 10''')
data

,genre,track_qty
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


#### **Rock**, disparado!

## Exiba os 5 clientes que mais fizeram pedidos.

In [ ]:
data = query('''SELECT c.id, first_name, last_name, count(quantity) total_qty
FROM customers c INNER JOIN invoices i on c.id = i.customer_id INNER JOIN invoice_lines il on i.id = il.invoice_id
GROUP BY c.id
ORDER BY total_qty desc''')

data

,id,first_name,last_name,total_qty
0,1,Luís,Gonçalves,38
1,2,Leonie,Köhler,38
2,3,François,Tremblay,38
3,4,Bjørn,Hansen,38
4,5,František,Wichterlová,38
5,6,Helena,Holý,38
6,7,Astrid,Gruber,38
7,8,Daan,Peeters,38
8,9,Kara,Nielsen,38
9,10,Eduardo,Martins,38


In [ ]:
print(show_schema('customers'))
print(show_schema('invoices'))
print(show_schema('invoice_lines'))

CREATE TABLE customers
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(40)  NOT NULL,
    last_name VARCHAR(20)  NOT NULL,
    company VARCHAR(80),
    address VARCHAR(70),
    city VARCHAR(40),
    state VARCHAR(40),
    country VARCHAR(40),
    postal_code VARCHAR(10),
    phone VARCHAR(24),
    fax VARCHAR(24),
    email VARCHAR(60)  NOT NULL,
    support_rep_id INTEGER,
    FOREIGN KEY (support_rep_id) REFERENCES employees (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)
None
CREATE TABLE invoices
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER  NOT NULL,
    invoice_date TIMESTAMP  NOT NULL,
    billing_address VARCHAR(70),
    billing_city VARCHAR(40),
    billing_state VARCHAR(40),
    billing_country VARCHAR(40),
    billing_postal_code VARCHAR(10),
    total NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)
None
CREATE TABLE invoice_lines
(
    id INTEGER PRIMA


## Quais o email dos 10 clientes que mais gastaram na loja?

In [ ]:
data = query('''SELECT first_name, last_name, email, total 
            FROM customers c 
            INNER JOIN invoices i ON c.id = i.customer_id''')

data

,first_name,last_name,email,total
0,Leonie,Köhler,leonekohler@surfeu.de,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,3.96
2,Daan,Peeters,daan_peeters@apple.be,5.94
3,Mark,Philips,mphilips12@shaw.ca,8.91
4,John,Gordon,johngordon22@yahoo.com,13.86
...,...,...,...,...
407,Victor,Stevens,vstevens@yahoo.com,3.96
408,Robert,Brown,robbrown@shaw.ca,5.94
409,Madalena,Sampaio,masampaio@sapo.pt,8.91
410,Terhi,Hämäläinen,terhi.hamalainen@apple.fi,13.86


In [ ]:
# top_spenders = data.groupby('email', as_index=False).agg({'total' : 'max'}).sort_values(by='total', ascending=False).reset_index(drop=True)
# top_spenders.head(10) ---> 1ª tentativa, mas só retorna o email...

teste = data.groupby(['first_name', 'last_name', 'email'], as_index=False).max('total').sort_values(by='total', ascending=False).reset_index(drop=True)
teste.head(15)

,first_name,last_name,email,total
0,Helena,Holý,hholy@gmail.com,25.86
1,Richard,Cunningham,ricunningham@hotmail.com,23.86
2,Ladislav,Kovács,ladislav_kovacs@apple.hu,21.86
3,Hugh,O'Reilly,hughoreilly@apple.ie,21.86
4,Astrid,Gruber,astrid.gruber@apple.at,18.86
5,Victor,Stevens,vstevens@yahoo.com,18.86
6,Luis,Rojas,luisrojas@yahoo.cl,17.91
7,František,Wichterlová,frantisekw@jetbrains.com,16.86
8,Isabelle,Mercier,isabelle_mercier@apple.fr,16.86
9,Frank,Ralston,fralston@gmail.com,15.86


In [ ]:
data1 = query('''SELECT first_name, last_name, email, max(total) gasto_total
            FROM customers c 
            INNER JOIN invoices i ON c.id = i.customer_id
            GROUP BY email
            ORDER BY gasto_total DESC LIMIT 10''')
data1

,first_name,last_name,email,gasto_total
0,Helena,Holý,hholy@gmail.com,25.86
1,Richard,Cunningham,ricunningham@hotmail.com,23.86
2,Hugh,O'Reilly,hughoreilly@apple.ie,21.86
3,Ladislav,Kovács,ladislav_kovacs@apple.hu,21.86
4,Astrid,Gruber,astrid.gruber@apple.at,18.86
5,Victor,Stevens,vstevens@yahoo.com,18.86
6,Luis,Rojas,luisrojas@yahoo.cl,17.91
7,František,Wichterlová,frantisekw@jetbrains.com,16.86
8,Isabelle,Mercier,isabelle_mercier@apple.fr,16.86
9,Bjørn,Hansen,bjorn.hansen@yahoo.no,15.86


### O 10º e 11º cliente gastaram a mesma quantidade

Qual o valor médio de cada compra?
Quantas compras estão acima da média?
Vamos fazer uma playlist com as 12 músicas mais vendidas! Quais foram elas? Quais os generos e os artistas?

## Qual o valor médio de cada compra?

In [ ]:
data = query()... 